In [1]:
# Installation des bibliothèques nécessaires
!pip install -U langchain langchain-community
!pip install langchain together
!pip install langgraph
!pip install evaluate
!pip install bert_score


In [2]:
# Importation des composants
from langchain.llms import Together
from langchain.prompts import PromptTemplate

import time
import pandas as pd
from google.colab import drive

from evaluate import load
from bert_score import score

from statistics import mean

from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages

In [3]:
# Initialisation du modèle de langage (LLM) via l'API Together
llm = Together(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    temperature=0.7,
    max_tokens=7000,
    together_api_key="1aa19d9568dc960ec217156fc5d5d657dae558d5707a3d8f1e99d67d60c128c4"
)


<ipython-input-3-691199504>:2: LangChainDeprecationWarning: The class `Together` was deprecated in LangChain 0.0.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-together package and should be used instead. To use it run `pip install -U :class:`~langchain-together` and import as `from :class:`~langchain_together import Together``.
  llm = Together(


In [4]:
# Liste d'exemples d'histoires pulp fiction (Datasets)
examples=[
     {
      "consigne": "Écris une histoire pulp fiction où un détective doit infiltrer un syndicat du crime chinois dans le quartier de Chinatown.",
      "histoire": "Les lanternes rouges de Chinatown projetaient leurs lueurs sanglantes sur les ruelles tortueuses de San Francisco, où Mike Chen, un détective sino-américain, se préparait à sa plus dangereuse mission. Engagé par une mystérieuse femme aux yeux de jade pour retrouver son frère disparu, Chen se retrouva rapidement plongé dans l'univers des Triades. Sous couverture comme joueur dans un casino clandestin, il découvrit que le Dragon Rouge, la plus puissante des Triades, orchestrait un vaste trafic d'antiquités volées. Mais plus il s'enfonçait dans ce monde de traditions ancestrales et de violence moderne, plus Chen réalisait que sa cliente n'était pas celle qu'elle prétendait être. Entre les rituels secrets des Triades et les vengeances familiales qui remontaient à trois générations, Chen devait naviguer dans un labyrinthe de loyautés où chaque faux pas pouvait être le dernier. Dans un final sanglant au temple du Dragon Rouge, Chen fut confronté à un choix impossible : trahir son héritage ou sacrifier sa mission."
    },
    {
      "consigne": "Écris une histoire pulp fiction sur un détective confronté à une série de meurtres rituels dans une ville industrielle en déclin.",
      "histoire": "Dans les ruines fumantes de Steel City, où les cheminées d'usines abandonnées crachaient encore leurs dernières volutes toxiques, l'inspecteur Sarah Blackwood suivait la piste d'un tueur méthodique. Les corps, disposés selon un motif ésotérique dans les vestiges des aciéries, portaient tous la même marque : un pentagramme gravé au fer rouge. Chaque nouvelle victime rapprochait Sarah d'une terrible vérité : les meurtres reproduisaient un rituel décrit dans un grimoire disparu depuis les années 30, époque où la ville était au sommet de sa gloire industrielle. Alors que les indices la menaient vers un cercle secret d'industriels pratiquant la magie noire pour restaurer la prospérité de la ville, Sarah découvrit que son propre grand-père, ancien magnat de l'acier, était impliqué dans ces rituels macabres. Dans les entrailles rouillées de la dernière aciérie encore debout, elle devrait affronter non seulement le tueur, mais aussi les démons du passé de sa famille."
    },
    {
      "consigne": "Écris une histoire pulp fiction mettant en scène un ancien flic devenu détective privé qui enquête sur la corruption au sein de son ancien département.",
      "histoire": "Les rues de Metro City n'avaient pas changé depuis que Jack Morrison avait rendu son badge, mais lui n'était plus le même homme. Trois ans après avoir été forcé de démissionner pour avoir dénoncé la corruption, une jeune recrue lui demanda de l'aide : des preuves disparaissaient mystérieusement des scellés, des témoins se rétractaient inexplicablement. En creusant l'affaire, Morrison découvrit un réseau tentaculaire dirigé par son ancien partenaire, devenu capitaine, qui monnayait la protection de la police auprès des cartels de drogue. Chaque indice le ramenait à cette nuit fatidique où il avait surpris des collègues en train de faire disparaître des preuves, la même nuit où sa femme avait eu un 'accident' de voiture. Armé de son appareil photo et de ses vieux contacts, Morrison s'enfonça dans les bas-fonds de la ville, où la frontière entre la loi et le crime s'effaçait dans la brume des néons. La vérité qu'il découvrit menaçait non seulement de détruire le département, mais aussi de révéler que la corruption remontait jusqu'aux plus hautes sphères de la ville."
    },
    {
      "consigne": "Écris une histoire pulp fiction où une détective traque un voleur d'œuvres d'art dans le monde des galeries huppées.",
      "histoire": "Les vernissages de Manhattan cachaient bien leur jeu, mais Elena Vasquez savait que derrière les sourires mondains et les coupes de champagne se tramait le plus grand coup du siècle. Une série de vols sophistiqués avait secoué le monde de l'art, chaque tableau dérobé étant remplacé par une copie parfaite qui n'était découverte que des mois plus tard. La piste la menait vers Augustus Vale, un restaurateur d'art réputé dont le charme n'avait d'égal que le mystère qui l'entourait. Plus Elena s'approchait de la vérité, plus elle se retrouvait fascinée par cet homme qui semblait danser sur la ligne entre l'art et le crime. Dans les sous-sols secrets des galeries, elle découvrit que les vols servaient à financer une opération bien plus sinistre : le blanchiment d'argent à l'échelle internationale à travers le marché de l'art. Le jeu du chat et de la souris entre Elena et Vale atteignit son paroxysme lors d'une vente aux enchères où des millions étaient en jeu, et où la frontière entre le chasseur et la proie devint aussi floue qu'une toile impressionniste."
    },
    {
      "consigne": "Écris une histoire pulp fiction sur un détective privé engagé pour retrouver l'héritier disparu d'une famille de la haute société.",
      "histoire": "Les gratte-ciels de Gold Coast projetaient leurs ombres démesurées sur les quartiers huppés où vivait la famille Blackwood, dynastie industrielle aux secrets aussi profonds que leurs coffres-forts. Ray Donovan, détective privé habitué aux affaires sordides, fut engagé par la matriarche pour retrouver son petit-fils Thomas, disparu depuis trois mois. L'enquête le mena des clubs privés aux repaires underground, où il découvrit que le jeune héritier menait une double vie, infiltrant des cercles anarchistes qui planifiaient de faire tomber l'empire familial. Mais plus Donovan creusait, plus il réalisait que Thomas n'était pas la victime : il était l'architecte d'un plan complexe visant à exposer les pratiques criminelles de sa propre famille. Entre les intrigues familiales dignes de Shakespeare et les complots d'entreprise, Donovan se retrouva pris dans un étau où chaque partie cherchait à l'utiliser. Dans les bureaux vertigineux de la Blackwood Tower, il devrait choisir entre la vérité et la justice, sachant que chaque option avait un prix mortel."
    }
  ]






In [5]:
# Exemple de prompt pour générer une histoire pulp fiction en français.
template=  """
Tu es un scénariste. Rédige une histoire pulp fiction en français à partir de la consigne suivante :

Consigne : {consigne}

Inspire-toi des exemples suivants pour adopter le même style et ton narratif :

{examples}

Histoire :

"""


In [7]:
# Création d'un prompt personnalisé à l'aide de PromptTemplate
prompt=PromptTemplate(template=template,
                             input_variable=["consigne"]).partial(examples= examples)

In [8]:
# Formatage du prompt avec une consigne spécifique
formatted_prompt = prompt.format_prompt(consigne="Crée une histoire pulp fiction pour enfants sur un lion et une grenouille. L’histoire doit avoir une vraie fin et ne pas dépasser 3000 mots.")

# Appel direct du modèle de langage (LLM) avec le prompt formaté
llm.invoke(formatted_prompt)

"Dans les savanes arides d'Outback, la lionne Mia et la grenouille François partageaient la vie la plus simple : chasser les animaux de la prairie et se reposer sous les arbres. Mais cette idylle soudaine se brisa avec l'arrivée du sorcier maléfique Zarko. Celui-ci, obsédé par sa vague ambition de dominer le monde, avait découvert une pierre magique perdue depuis des millénaires : la Pierre de la Vie. Pour l'utiliser, il avait besoin de l'aide d'un serviteur puissant : et c'était pourquoi il avait demandé à la Grenouille François de l'appeler.\n\nMais la Grenouille François était une grenouille ordinaire, sans pouvoir magique. Pour l'aider, Mia et François décidèrent de l'aider à trouver la pierre. À travers les étendues désertiques, ils durent affronter de nombreux obstacles : des lapins géants, des serpents venimeux, des rongeurs énormes. Mais la plus grande difficulté vint de Zarko lui-même : il avait le pouvoir de se transformer en toutes sortes d'animaux, et il essayait à chaque f

In [9]:
# Génére une histoire à partir d’une consigne avec prompt personnalisé
def generation_histoire_avec_prompt(consigne):
    try:
        formatted_prompt = prompt.format_prompt(consigne=consigne)
        result = llm.invoke(formatted_prompt)
        time.sleep(1)
        if result is not None:
            return result
        else:
            return ""
    except Exception as e:
        print(f"Error processing consigne: {consigne[:50]}... Error: {e}")
        return f"ERROR: {e}"


In [10]:
# Montage de Google Drive dans l'environnement Colab pour accéder aux fichiers stockés dans le Drive.
drive.mount('/content/drive')


Mounted at /content/drive


In [11]:
# Type de génération : avec prompt + exemples (style pulp fiction)
# Chargement du fichier Excel
consignes_avec_prompt  = pd.read_excel('/content/drive/My Drive/data.xlsx')
# Génère une histoire à chaque ligne de la colonne 'consigne' du DataFrame `data`.
consignes_avec_prompt['histoire_model'] = consignes_avec_prompt['consigne'].apply(generation_histoire_avec_prompt)
# Sauvegarde du DataFrame `data` dans un fichier Excel
consignes_avec_prompt.to_excel('/content/drive/My Drive/consignes_avec_prompt.xlsx', index=False)

Error processing consigne: Écris-moi une histoire  sur une femme fatale en ca... Error: Together Server: Error 500
Error processing consigne: Imagine une histoire pulp fiction dans le style de... Error: Together Server: Error 500


In [12]:
# Génére une histoire simplement à partir d’une consigne brute (sans prompt)
def generer_histoire_sans_prompt(consigne):
    try:
        result = llm.invoke(consigne)
        time.sleep(1)
        if result is not None:
            return result
        else:
            return ""
    except Exception as e:
        print(f"Error processing consigne: {consigne[:50]}... Error: {e}")
        return f"ERROR: {e}"

In [13]:
# Type de génération : brute (sans style, juste la consigne)
consignes_sans_prompt = pd.read_excel('/content/drive/My Drive/data1.xlsx')
consignes_sans_prompt ['histoire_model_brut'] = consignes_sans_prompt ['consigne'].apply(generer_histoire_sans_prompt)
consignes_sans_prompt .to_excel('/content/drive/My Drive/consignes_sans_prompt.xlsx', index=False )

Error processing consigne: Génère une histoire pulp fiction dans le style noi... Error: Together Server: Error 500


In [29]:
# Chargement de la métrique BERTScore
bertscore = load("bertscore")

In [30]:
# Chargement du fichier Excel dans un DataFrame Pandas
evaluation_avec_prompt= pd.read_excel('/content/drive/My Drive/consignes_avec_prompt.xlsx')

# Extraction des colonnes 'histoire_model' et 'histoire' du DataFrame `evaluation`
predictions_avec_prompt = evaluation_avec_prompt["histoire_model"].astype(str).tolist()
references_avec_prompt = evaluation_avec_prompt["histoire"].astype(str).tolist()

# Calcul des scores BERTScore entre les textes générés (predictions) et les textes de référence (references)
results_avec_prompt = bertscore.compute(predictions=predictions_avec_prompt, references=references_avec_prompt, lang="fr")

In [31]:
results_avec_prompt["f1"]

[0.6875433325767517,
 0.545774519443512,
 0.6596840620040894,
 0.6814069747924805,
 0.525589644908905,
 0.692867636680603,
 0.7153210043907166,
 0.6583645343780518,
 0.6849310994148254,
 0.6679114699363708]

In [32]:
results_avec_prompt["recall"]

[0.7319057583808899,
 0.525109052658081,
 0.699839174747467,
 0.719702959060669,
 0.4944773018360138,
 0.7147614359855652,
 0.7276340126991272,
 0.6647535562515259,
 0.7132907509803772,
 0.6890534162521362]

In [33]:
results_avec_prompt["precision"]

[0.6482514142990112,
 0.5681331157684326,
 0.6238868832588196,
 0.6469807028770447,
 0.5608800053596497,
 0.6722752451896667,
 0.7034177780151367,
 0.6520971059799194,
 0.6587402820587158,
 0.6480283141136169]

In [34]:
evaluation_sans_prompt= pd.read_excel('/content/drive/My Drive/consignes_sans_prompt.xlsx')
predictions_sans_prompt = evaluation_sans_prompt["histoire_model_brut"].astype(str).tolist()
references_sans_prompt = evaluation_sans_prompt["histoire"].astype(str).tolist()
results_sans_prompt = bertscore.compute(predictions=predictions_sans_prompt, references=references_sans_prompt, lang="fr")

In [35]:
results_sans_prompt["f1"]

[0.6389510035514832,
 0.6816247701644897,
 0.6565691828727722,
 0.6631003022193909,
 0.650140106678009,
 0.5912732481956482,
 0.5106707811355591,
 0.4993896186351776,
 0.5801318287849426]

In [36]:
results_sans_prompt["recall"]

[0.641130268573761,
 0.6763982772827148,
 0.6810842752456665,
 0.6728613376617432,
 0.6565331220626831,
 0.600677490234375,
 0.4684237837791443,
 0.5294826030731201,
 0.6147037744522095]

In [37]:
results_sans_prompt["precision"]

[0.6367865204811096,
 0.6869326233863831,
 0.6337575912475586,
 0.6536185145378113,
 0.6438703536987305,
 0.5821589231491089,
 0.5612936615943909,
 0.4725334048271179,
 0.5492415428161621]

In [46]:
# Calcul de la moyenne des scores F1 BERTScore pour deux ensembles de résultats
f1_sans_prompt = mean(results_sans_prompt['f1'])
f1_avec_prompt = mean(results_avec_prompt['f1'])

In [43]:
f1_avec_prompt

0.6519394278526306

In [44]:
f1_sans_prompt

0.6079834269152747

In [45]:
# Calcul de la différence relative absolue entre les deux moyennes de scores F1
abs(f1_avec_prompt - f1_sans_prompt) / f1_sans_prompt

0.07229802489909207

In [47]:
precision_sans_prompt = mean(results_sans_prompt['precision'])
precision_avec_prompt = mean(results_avec_prompt['precision'])

abs(precision_avec_prompt - precision_sans_prompt) / precision_sans_prompt

0.05981864822340337

In [48]:
recall_sans_prompt = mean(results_sans_prompt['recall'])
recall_avec_prompt = mean(results_avec_prompt['recall'])

abs(recall_avec_prompt - recall_sans_prompt) / recall_sans_prompt

0.08503062016230152

In [49]:
# Définition d’un type dictionnaire nommé `State` avec TypedDict
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Construction d’un graphe d’état
graph_builder = StateGraph(State)


In [50]:
# Prend l’état actuel contenant l’historique des messages et appelle le modèle LLM avec
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

In [51]:
# Connexion du graphe
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()

In [55]:
# Fonction qui gère le streaming des mises à jour du graphe conversationnel
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


while True:
    user_input = input("Utilisateur: ")

    if user_input.lower() in ["quit", "exit", "q"]:
        print("A bientôt!")
        break

    try:
        histoire = generation_histoire_avec_prompt(user_input)
        print("Assistant: Voici l'histoire :")
        print(histoire)

        feedback = input("Êtes-vous satisfait de l'histoire? (oui/non) : ")
        if feedback.lower() == "oui":
            print("A bientôt!")
            break
        elif feedback.lower() == "non":
            histoire = generation_histoire_avec_prompt(user_input)
            print("Assistant: Voici une nouvelle histoire :")
            print(histoire)
        else:
            print("Réponse non reconnue, veuillez répondre par 'oui' ou 'non'.")

    except Exception as e:
        print(f"Une erreur est survenue : {e}. Veuillez réessayer.")

Utilisateur: Créé une histoire avec des princesses
Assistant: Voici l'histoire :
Dans les rues de Paris, où les lanternes des Halles projetaient leurs lueurs dans les ruelles tortueuses du quartier latin, Léo Dupont, un détective français, se préparait à sa plus dangereuse mission. Engagé par une mystérieuse princesse égyptienne pour retrouver son frère disparu, Dupont se retrouva rapidement plongé dans l'univers des sorcières. Sous couverture comme étudiant dans une école de sorcellerie, il découvrit que la sorcière blanche, la plus puissante des sorcières, orchestrait un vaste trafic de secrets magiques. Mais plus il s'enfonçait dans ce monde de traditions ancestrales et de sorcellerie moderne, plus Dupont réalisait que sa cliente n'était pas celle qu'elle prétendait être. Entre les rituels secrets des sorcières et les vengeances familiales qui remontaient à trois générations, Dupont devait naviguer dans un labyrinthe de loyautés où chaque faux pas pouvait être le dernier. Dans un fi